In [1]:
%matplotlib inline
%load_ext tensorboard

import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

2024-04-29 16:53:04.031282: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Data Cleaning

In [2]:
data = pd.read_csv('data_file.csv') # pandas data frame

/tmp/ipykernel_7478/3928255717.py:1: DtypeWarning: Columns (45,179,180,182,183,189,190,193,194,197,198,203,204,205,206,209,210,213,214,218,219,220,222,224,226,233,234,235,236,237,238,248,249,253,254,255,260,262,263,266,267,268,269,283,284,294,295,301) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data_file.csv') # pandas data frame


In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# data.head(100)

In [4]:
# Removes 
data = data.loc[(data['desc'].str.startswith('('))]
data = data[data['play_type'] != 'qb_kneel']
data = data[data['play_type'] != 'no_play']

##data = data[data['posteam'] == 'NE']

data = data.dropna(subset=['down'])
data = data.dropna(subset=['play_type'])
data = data.dropna(subset=['wp'])
data = data[(data['play_type'] == 'pass') | (data['play_type'] == 'run') | (data['play_type'] == 'field_goal') | (data['play_type'] == 'punt')]

# Extract year and month using string slicing
data['year'] = data['game_id'].str[:4].astype(int)
data['month'] = data['game_id'].str[5:7].astype(int)
 
# Fixes playtype to be strictly pass or run 
# Lables scrambles as passes
data.loc[data['pass']==1,'play_type'] = 'pass'
data.loc[data.rush==1,'play_type'] = 'run'

columnsKeep = ['game_id', 'score_differential_post','year','month','season_type','week','posteam','posteam_type','defteam','side_of_field','yardline_100','half_seconds_remaining','home_team','away_team',
               'game_seconds_remaining','game_half','drive','down','time','yrdln','ydstogo','desc','play_type','yards_gained','shotgun','no_huddle','qb_dropback','qb_kneel',
               'qb_spike','qb_scramble','pass_length','run_location','run_gap','home_timeouts_remaining','away_timeouts_remaining','total_home_score','total_away_score',
               'passer_player_name','receiver_player_name','rusher_player_name','play_type_nfl','roof','surface','temp','wind','home_coach','away_coach','game_stadium',
               'success', 'wp']

data = data[columnsKeep]

In [5]:
# Function to check if a value is a number or not
def is_not_number(val):
    return not isinstance(val, (int, float, complex))

In [6]:
# Creates a variable for whether the coach is home or away
def pos_coach(row):
    if row['posteam_type'] == 'home':
        return f'{row["home_coach"]}'
    elif row['posteam_type'] == 'away':
        return f'{row["away_coach"]}'
    
data['pos_coach'] = data.apply(pos_coach,axis=1)

In [7]:
# Encode coaches to a number
le = LabelEncoder()
data['pos_coach'] = le.fit_transform(data['pos_coach'])

In [8]:
print(len(data))

# Count occurrences of each play type
runCount = (data['play_type'] == 'run').sum()
passCount = (data['play_type'] == 'pass').sum()
fgCount = (data['play_type'] == 'field_goal').sum()
puntCount = (data['play_type'] == 'punt').sum()


print("Number of 'run' plays:", runCount)
print("Number of 'pass' plays:", passCount)
print("Number of 'fg' plays:", fgCount)
print("Number of 'punt' plays:", puntCount)



334503
Number of 'run' plays: 117123
Number of 'pass' plays: 186390
Number of 'fg' plays: 9285
Number of 'punt' plays: 21705


In [9]:
# Check for nan values in data
for index, row in data.iterrows():
    if pd.isnull(row['play_type']):  # Check if the value in the 'down' column is NaN
        print(row['desc'])  # Print the value in the 'desc' column if 'down' is NaN

# Wind and stadium had a lot of null 
# Check for missing values in data
selected_features = ['down','pos_coach', 'score_differential_post','drive', 'ydstogo', 'yardline_100', 'half_seconds_remaining', 'game_seconds_remaining','wp','week','yards_gained']
missing_values_features = data[selected_features].isnull().sum()
print("Missing values in selected features:")
print(missing_values_features)

# Check for missing values in the target
missing_values_target = data['play_type'].isnull().sum()
print("\nMissing values in target (play_type):", missing_values_target)


Missing values in selected features:
down                       0
pos_coach                  0
score_differential_post    0
drive                      0
ydstogo                    0
yardline_100               0
half_seconds_remaining     0
game_seconds_remaining     0
wp                         0
week                       0
yards_gained               0
dtype: int64

Missing values in target (play_type): 0


In [10]:
features = data[['down','pos_coach','score_differential_post','drive', 'ydstogo','yardline_100', 'half_seconds_remaining', 'game_seconds_remaining', 'wp']]
target = data['play_type'].map({'pass': 0, 'run': 1, 'field_goal':2,'punt':3})

# Initialize the scaler 
scaler = StandardScaler() 

# Fit the scaler on the features and transform them 
features_scaled = scaler.fit_transform(features)

# Reshaping input data for LSTM [samples, time steps, features]
# Here, we assume each sample is a single time step
features_scaled = features_scaled.reshape((features_scaled.shape[0], 1, features_scaled.shape[1]))

# 1 indicates that each sample consists of 1 timestep. This is a key detail: 
# it means you are treating each play as a separate, single-timestep sequence.
# This is a simplification, as ideally, plays within a single drive might be treated as sequences to capture their temporal dependencies.

X_train, X_test, y_train, y_test = train_test_split(features_scaled, target, test_size=0.2, shuffle = False)

In [11]:
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    LSTM(32,return_sequences=False),
    Dense(4, activation='softmax')
])

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=Adam(learning_rate=0.001),
    metrics=['accuracy']
)

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.1)


test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc:.3f}, Test Loss: {test_loss:.3f}')

2024-04-29 16:54:06.804245: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2024-04-29 16:54:07.074526: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-29 16:54:07.076442: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-29 16:54:07.0

Epoch 1/10


2024-04-29 16:54:07.751845: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-29 16:54:07.754267: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-29 16:54:07.756304: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

3764/3764 [==============================] - ETA: 0s - loss: 0.5832 - accuracy: 0.6958

2024-04-29 16:56:09.004647: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-29 16:56:09.008230: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-29 16:56:09.012470: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

3764/3764 [==============================] - 127s 32ms/step - loss: 0.5832 - accuracy: 0.6958 - val_loss: 0.5374 - val_accuracy: 0.7064
Epoch 2/10
3764/3764 [==============================] - 34s 9ms/step - loss: 0.5316 - accuracy: 0.7154 - val_loss: 0.5247 - val_accuracy: 0.7202
Epoch 3/10
3764/3764 [==============================] - 35s 9ms/step - loss: 0.5235 - accuracy: 0.7191 - val_loss: 0.5216 - val_accuracy: 0.7199
Epoch 4/10
3764/3764 [==============================] - 39s 10ms/step - loss: 0.5191 - accuracy: 0.7204 - val_loss: 0.5185 - val_accuracy: 0.7225
Epoch 5/10
3764/3764 [==============================] - 40s 11ms/step - loss: 0.5168 - accuracy: 0.7214 - val_loss: 0.5138 - val_accuracy: 0.7250
Epoch 6/10
3764/3764 [==============================] - 44s 12ms/step - loss: 0.5146 - accuracy: 0.7224 - val_loss: 0.5145 - val_accuracy: 0.7240
Epoch 7/10
3764/3764 [==============================] - 50s 13ms/step - loss: 0.5127 - accuracy: 0.7235 - val_loss: 0.5155 - val_accurac

In [12]:

predictions = model.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1)

from sklearn.metrics import classification_report, confusion_matrix

print("Classification Report:")
print(classification_report(y_test, predicted_classes))

print("Confusion Matrix:")
print(confusion_matrix(y_test, predicted_classes))


2024-04-29 17:03:56.082757: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-29 17:03:56.084103: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-29 17:03:56.085302: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2091/2091 [==============================] - 13s 6ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.79      0.77     37737
           1       0.64      0.56      0.59     23520
           2       0.86      0.86      0.86      1858
           3       0.91      0.99      0.95      3786

    accuracy                           0.72     66901
   macro avg       0.79      0.80      0.79     66901
weighted avg       0.72      0.72      0.72     66901

Confusion Matrix:
[[29893  7468   160   216]
 [10187 13116    90   127]
 [  187    32  1599    40]
 [   17     7    19  3743]]


In [13]:
dummies = pd.get_dummies(data['play_type'], prefix='play_type')

df = pd.concat([data, dummies], axis=1)

In [14]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))

corr_matrix = df.corr()

sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', cbar=True)

plt.show()

ValueError: could not convert string to float: '2012_01_ATL_KC'

<Figure size 800x600 with 0 Axes>

In [ ]:
# Length of X train, and nulls in the x,train, x_test, y_train, y_test 
print(len(X_train))
nan_indices_X = np.isnan(X_train)
nan_count_X = np.sum(nan_indices_X)
print(nan_count_X)

nan_indices_X_test = np.isnan(X_test)
nan_count_X_test = np.sum(nan_indices_X_test)
print(nan_count_X_test)

nan_indices_y = np.isnan(y_train)
nan_count_y = np.sum(nan_indices_y)
print(nan_count_y)

nan_indices_Y_test = np.isnan(y_test)
nan_count_Y_test = np.sum(nan_indices_Y_test)
print(nan_count_Y_test)

print("Shape of X_train:", X_train.shape)

739764
0
0
0
0
Shape of X_train: (739764, 1, 7)
